<a href="https://colab.research.google.com/github/Maxwell-Tuttle/COS470/blob/main/NLP_Assignment2_MaxTuttle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
from post_parser_record import PostParserRecord
from collections import Counter

## Getting the top-20 frequent tags in LawSE -- There is a reason for passing 21
def get_frequent_tags(post_parser, topk=21):
  lst_tags = []
  for question_id in post_parser.map_questions:
    question = post_parser.map_questions[question_id]
    creation_date_year = int(question.creation_date.split("-")[0])
    tag = question.tags[0]
    lst_tags.append(tag)
  tag_freq_dic = dict(Counter(lst_tags))
  tag_freq_dic = dict(sorted(tag_freq_dic.items(), key=lambda item: item[1], reverse=True))
  return list(tag_freq_dic.keys())[:topk]

In [15]:
# Getting dictionary of train and test samples in form of
# key: tag value: list of tuples in form of (title, body)
def build_train_test(post_parser, lst_frequent_tags):
  dic_training = {}
  dic_test = {}
  for question_id in post_parser.map_questions:
    question = post_parser.map_questions[question_id]
    creation_date_year = int(question.creation_date.split("-")[0])
    tag = question.tags[0]
    if tag in lst_frequent_tags:
      title = question.title
      body = question.body
      if creation_date_year > 2021:
        if tag in dic_test:
          dic_test[tag].append((title, body))
        else:
          dic_test[tag] = [(title, body)]
      else:
        if tag in dic_training:
          dic_training[tag].append((title, body))
        else:
          dic_training[tag] = [(title, body)]
  return dic_test, dic_training

In [16]:
post_parser = PostParserRecord("Posts_law.xml")
lst_frequent_tags = get_frequent_tags(post_parser)
# We removed contract as it had no post after 2021
lst_frequent_tags.remove("contract")
dic_test, dic_training = build_train_test(post_parser, lst_frequent_tags)
print("class\t#training\t#test")
for item in dic_training:
  print(str(item) + "\t" +str(len(dic_training[item]))+"\t"+str(len(dic_test[item])))

class	#training	#test
criminal-law	948	78
copyright	2016	181
united-states	5668	863
united-kingdom	1195	271
employment	238	36
international	316	43
canada	382	35
intellectual-property	301	29
england-and-wales	165	138
european-union	219	30
licensing	241	29
california	391	41
internet	416	39
business	171	7
rental-property	158	20
software	292	33
contract-law	1065	111
privacy	351	23
constitutional-law	177	21
gdpr	435	63


In [17]:
# At this point you have your training and test samples. Time to Implement your classifier

In [63]:
import re
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
import numpy as np

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [126]:
class Naive_Bayes_Classifier: 

  def __init__(self, input):
    self.input = input

  def tokenize(self, dic):
    all_words = []
    dictionary_tag_words = {}

    for i in dic: 
      #print(i)
      #print(len(dic[i]))
      #all_questions = []
      lst_sentences_by_tag = dic[i]
      words_per_tag = []
      for j in lst_sentences_by_tag:
        if self.input == "Titles":
          questions = j[0]
          #all_questions.append(questions)

        elif self.input == "Bodies":
          questions = j[1]

        elif self.input == "Titles and Bodies":
          questions = j[0] + j[1]


        tokenizer = nltk.RegexpTokenizer(r'\w+')
        for word in tokenizer.tokenize(questions):
          words_per_tag.append(word.lower())
          
         
      
      all_words.extend(words_per_tag)
      dictionary_tag_words[i] = dict(Counter(words_per_tag))

    vocab = dict(Counter(all_words))
    print("*************************************************************************")
    return vocab, dictionary_tag_words#, all_questions


  def create_feature_matrix(self, dic_words):
    #Create set of unique words
    word_set = set()
    #tag_list = []
    for tag in dic_words.keys():
      #tag_list.append(tag)
      for word in dic_words[tag].keys():
        word_set.add(word)

    #Create a dictionary to map words to their indices
    word_dic = {}
    for i, word in enumerate(word_set):
      word_dic[word] = i

    #Create a feature matrix
    X = np.zeros((len(dic_words.keys()), len(word_set)))
    for i, tag in enumerate(dic_words.keys()):
      for word in dic_words[tag].keys():
        if word in word_dic:
          X[i, word_dic[word]] += 1

    word_list = list(word_set)
    return X, word_list

  def fit(X,y):
    #Calculate class priors
    class_priors = np.zeros(2)
    for i in range(len(y)):
        class_priors[y[i]] += 1
    class_priors /= len(y)

    #Calculate class conditional probabilities
    class_counts = np.zeros((2, X.shape[1]))
    for i in range(len(y)):
        class_counts[y[i]] += X[i]
    class_probs = (class_counts + 1) / (np.sum(class_counts, axis=1, keepdims=True) + X.shape[1])

    return class_priors, class_probs

In [127]:
NBC_Titles_Training = Naive_Bayes_Classifier("Titles")
vocab, dictionary_tag_words = NBC_Titles_Training.tokenize(dic_training)

*************************************************************************


In [135]:
X, word_list = NBC_Titles_Training.create_feature_matrix(dictionary_tag_words)

In [136]:
X = X.astype(bool)

In [137]:
X

array([[False, False, False, ..., False, False,  True],
       [False,  True, False, ..., False,  True,  True],
       [ True,  True,  True, ..., False,  True,  True],
       ...,
       [False, False,  True, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False,  True, False]])

In [129]:
word_list

['constituent',
 'belong',
 'backyard',
 'lawyering',
 'pandemic',
 'rural',
 'million',
 'encrochat',
 'venue',
 'blues',
 'sandwiches',
 'prosus',
 'nannies',
 'excluded',
 'interstate',
 'room',
 'cuban',
 'donoghue',
 'libre',
 'forwarded',
 'museum',
 'epstein',
 'codebase',
 'ordernumber',
 'racist',
 'relevant',
 'rss',
 'statutory',
 'unstructured',
 'sever',
 'was',
 'entrance',
 'thru',
 'inherit',
 'outside',
 'arrangements',
 'sergeant',
 'compacts',
 'cooperation',
 'constitutions',
 'worm',
 'whole',
 'documented',
 'weapons',
 'jammu',
 'jenkins',
 'prosecute',
 'mountain',
 'droid',
 'yanez',
 'asking',
 'donation',
 'chalk',
 'limitations',
 'enemy',
 'continue',
 'eva',
 'organizer',
 'opposed',
 'miami',
 'kjv',
 'object',
 'injecting',
 'unwarranted',
 'paperwork',
 'sees',
 'synthesis',
 'exe',
 'methodology',
 'keygen',
 'wechat',
 'successfully',
 'termination',
 'equivalence',
 'irs',
 'jo',
 'diversifying',
 'victim',
 'auto',
 'furry',
 'bait',
 '25th',
 'perm

In [138]:
class_priors, class_probs = NBC_Titles_Training.fit(X)

IndexError: ignored

In [32]:
dictionary_tag_words

{'criminal-law': {'are': 73,
  'there': 53,
  'any': 31,
  'kinds': 3,
  'of': 249,
  'laws': 38,
  'that': 57,
  'prohibit': 1,
  'personally': 1,
  'harmful': 2,
  'speech': 1,
  'does': 63,
  'killing': 5,
  'witnesses': 5,
  'before': 13,
  'trial': 20,
  'suppress': 1,
  'them': 8,
  'is': 306,
  'way': 6,
  'to': 363,
  'bring': 2,
  'a': 511,
  'criminal': 93,
  'case': 29,
  'the': 356,
  'grand': 2,
  'jury': 14,
  'without': 32,
  'support': 4,
  'prosecuting': 3,
  'attorney': 9,
  'marital': 1,
  'rape': 8,
  'as': 32,
  'defined': 2,
  'by': 28,
  'indian': 3,
  'law': 71,
  'what': 135,
  'difference': 9,
  'between': 16,
  'murder': 26,
  'manslaughter': 6,
  'and': 123,
  'homicide': 5,
  'furtherance': 1,
  'crime': 58,
  'exception': 1,
  'client': 10,
  'privilege': 2,
  'apply': 13,
  'if': 105,
  'unsure': 1,
  'happens': 11,
  'indonesian': 1,
  'constitutional': 4,
  'court': 36,
  'rules': 4,
  'in': 268,
  'favor': 2,
  'an': 97,
  'executed': 1,
  'person': 42

criminal-law
copyright
united-states
united-kingdom
employment
international
canada
intellectual-property
england-and-wales
european-union
licensing
california
internet
business
rental-property
software
contract-law
privacy
constitutional-law
gdpr


In [23]:
NBC_Titles_Test = Naive_Bayes_Classifier("Titles")
NBC_Titles_Test.tokenize(dic_test)

*************************************************************************


({'what': 326,
  'stops': 1,
  'states': 24,
  'from': 136,
  'creating': 10,
  'money': 22,
  'out': 37,
  'of': 629,
  'thin': 1,
  'air': 2,
  'bank': 10,
  'will': 26,
  'not': 111,
  'take': 23,
  'check': 5,
  'squatters': 1,
  'are': 217,
  'trespassers': 1,
  'in': 556,
  'california': 27,
  'claims': 15,
  'may': 10,
  'state': 73,
  'a': 1138,
  'dollar': 2,
  'amount': 7,
  'complaint': 5,
  'or': 148,
  'its': 21,
  'prayer': 2,
  'judges': 5,
  'allowed': 23,
  'to': 943,
  'award': 6,
  'punitive': 3,
  'damages': 10,
  'totaling': 1,
  'an': 235,
  'higher': 5,
  'than': 32,
  'the': 946,
  'jurisdictional': 1,
  'limit': 9,
  'small': 9,
  'case': 40,
  'legal': 245,
  'stupid': 1,
  'behavior': 1,
  'how': 148,
  'can': 355,
  'you': 127,
  'reference': 5,
  'fictitious': 1,
  'business': 30,
  'on': 194,
  'front': 2,
  'page': 3,
  'is': 641,
  'it': 296,
  'say': 14,
  'i': 189,
  'live': 7,
  'at': 37,
  'another': 34,
  'address': 15,
  'medi': 1,
  'cal': 1,
  'e

In [24]:
NBC_Bodies_Training = Naive_Bayes_Classifier("Bodies")
NBC_Bodies_Training.tokenize(dic_training)

*************************************************************************


({'p': 118038,
  'in': 40364,
  'response': 298,
  'to': 75239,
  'a': 82135,
  'href': 8294,
  'http': 1628,
  'www': 4063,
  'economist': 8,
  'com': 5622,
  'news': 818,
  'americas': 7,
  '21651849': 1,
  'crackdown': 3,
  'public': 2111,
  'displays': 41,
  'machismo': 1,
  'criminalising': 4,
  'catcallers': 1,
  'rel': 7066,
  'nofollow': 6280,
  'this': 21657,
  'article': 1209,
  'from': 10078,
  'the': 125798,
  'i': 52181,
  'was': 10703,
  'wondering': 763,
  'if': 16334,
  'there': 8566,
  'are': 12833,
  'any': 9525,
  'other': 5480,
  'countries': 613,
  'that': 35421,
  'prohibit': 144,
  'type': 420,
  'of': 51307,
  'speech': 280,
  'not': 17803,
  'libel': 74,
  'since': 1974,
  'relates': 76,
  'directly': 507,
  'issues': 528,
  'right': 2323,
  'good': 767,
  'name': 2023,
  'nor': 443,
  'kinds': 93,
  'blasphemy': 8,
  'or': 21366,
  'profanity': 10,
  'laws': 2084,
  'rather': 629,
  'what': 9209,
  'm': 4962,
  'looking': 611,
  'for': 26690,
  'protect': 479,

In [25]:
NBC_Bodies_Test = Naive_Bayes_Classifier("Bodies")
NBC_Bodies_Test.tokenize(dic_test)

*************************************************************************


({'p': 15042,
  'when': 526,
  'a': 11202,
  'private': 129,
  'bank': 73,
  'creates': 23,
  'money': 191,
  'it': 3449,
  'obeys': 3,
  'reserve': 7,
  'accounting': 4,
  'and': 6024,
  'can': 1401,
  't': 1110,
  'create': 107,
  'more': 393,
  'but': 1447,
  'i': 5292,
  'get': 326,
  'paid': 143,
  'from': 1355,
  'the': 16500,
  'state': 471,
  'of': 6901,
  'california': 129,
  'comes': 54,
  'their': 981,
  'treasury': 17,
  'account': 145,
  'isn': 104,
  'that': 4703,
  'has': 907,
  'regulation': 48,
  'or': 2833,
  'anything': 193,
  'what': 1224,
  'stops': 7,
  'creating': 42,
  'infinite': 2,
  'free': 168,
  'its': 251,
  'accounts': 34,
  's': 1978,
  'not': 2265,
  'like': 595,
  'they': 1605,
  'open': 134,
  'chase': 4,
  'are': 1767,
  'limited': 48,
  'by': 1498,
  'balance': 14,
  'am': 584,
  'trying': 79,
  'to': 9690,
  'liquidate': 4,
  'everything': 47,
  'escape': 7,
  'coming': 21,
  'market': 41,
  'collapse': 2,
  'went': 37,
  'with': 1634,
  '600k': 1,

In [26]:
NBC_Titles_and_Bodies_Training = Naive_Bayes_Classifier("Titles and Bodies")
NBC_Titles_and_Bodies_Training.tokenize(dic_training)

*************************************************************************


({'are': 13964,
  'there': 9076,
  'any': 9939,
  'kinds': 101,
  'of': 55426,
  'laws': 2426,
  'that': 36277,
  'prohibit': 167,
  'personally': 232,
  'harmful': 24,
  'speech': 320,
  'p': 118041,
  'in': 44249,
  'response': 311,
  'to': 81253,
  'a': 90036,
  'href': 8294,
  'http': 1630,
  'www': 4064,
  'economist': 9,
  'com': 5639,
  'news': 839,
  'americas': 8,
  '21651849': 1,
  'crackdown': 3,
  'public': 2453,
  'displays': 42,
  'machismo': 1,
  'criminalising': 4,
  'catcallers': 1,
  'rel': 7066,
  'nofollow': 6280,
  'this': 21973,
  'article': 1261,
  'from': 11036,
  'the': 131758,
  'i': 54220,
  'was': 10952,
  'wondering': 764,
  'if': 17721,
  'other': 5751,
  'countries': 682,
  'type': 445,
  'not': 18504,
  'libel': 97,
  'since': 1987,
  'relates': 79,
  'directly': 521,
  'issues': 577,
  'right': 2568,
  'good': 801,
  'name': 2245,
  'nor': 445,
  'blasphemy': 10,
  'or': 22212,
  'profanity': 12,
  'rather': 642,
  'what': 11106,
  'm': 5020,
  'looking

In [27]:
NBC_Titles_and_Bodies_Test = Naive_Bayes_Classifier("Titles and Bodies")
NBC_Titles_and_Bodies_Test.tokenize(dic_test)

*************************************************************************


({'what': 1550,
  'stops': 8,
  'states': 388,
  'from': 1491,
  'creating': 52,
  'money': 213,
  'out': 448,
  'of': 7530,
  'thin': 1,
  'air': 13,
  'p': 15042,
  'when': 635,
  'a': 12340,
  'private': 146,
  'bank': 83,
  'creates': 24,
  'it': 3745,
  'obeys': 3,
  'reserve': 9,
  'accounting': 5,
  'and': 6321,
  'can': 1756,
  't': 1166,
  'create': 119,
  'more': 404,
  'but': 1476,
  'i': 5481,
  'get': 348,
  'paid': 159,
  'the': 17446,
  'state': 544,
  'california': 156,
  'comes': 56,
  'their': 1036,
  'treasury': 18,
  'account': 163,
  'isn': 111,
  'that': 4857,
  'has': 934,
  'regulation': 53,
  'or': 2981,
  'anything': 202,
  'infinite': 2,
  'free': 188,
  'its': 272,
  'accounts': 40,
  's': 2156,
  'not': 2376,
  'like': 607,
  'they': 1660,
  'open': 149,
  'chase': 4,
  'are': 1984,
  'limited': 50,
  'by': 1637,
  'balance': 15,
  'will': 625,
  'take': 273,
  'check': 65,
  'am': 601,
  'trying': 85,
  'to': 10633,
  'liquidate': 4,
  'everything': 50,
  